In [1]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("input"))
import warnings
warnings.filterwarnings('ignore')

['IMDB Dataset.csv']


In [11]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "ml-spec-0488fd916a4d.json"

In [12]:
imdb_data=pd.read_csv('input/IMDB Dataset.csv')
print(imdb_data.shape)

(50000, 2)


In [13]:
imdb_data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [14]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


In [15]:
imdb_data['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [26]:
imdb_data['sentiment_code'] =  [1 if x == "positive" else 0 for x in imdb_data["sentiment"]]

In [27]:
len(imdb_data['sentiment_code'])

50000

In [57]:
from google.cloud import language_v2


def sample_analyze_sentiment(text_content: str = "I am so happy and joyful.") -> None:
    """
    Analyzes Sentiment in a string.

    Args:
        text_content: The text content to analyze.
    """

    client = language_v2.LanguageServiceClient()

    # text_content = 'I am so happy and joyful.'

    # Available types: PLAIN_TEXT, HTML
    document_type_in_plain_text = language_v2.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language_code = "en"
    document = {
        "content": text_content,
        "type_": document_type_in_plain_text,
        "language_code": language_code,
    }

    # Available values: NONE, UTF8, UTF16, UTF32
    # See https://cloud.google.com/natural-language/docs/reference/rest/v2/EncodingType.
    encoding_type = language_v2.EncodingType.UTF8

    response = client.analyze_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )
    # Get overall sentiment of the input document
    #print(f"Document sentiment score: {response.document_sentiment.score}")
    #print(f"Document sentiment magnitude: {response.document_sentiment.magnitude}")
    # Get sentiment for all sentences in the document
    #for sentence in response.sentences:
        #print(f"Sentence text: {sentence.text.content}")
        #print(f"Sentence sentiment score: {sentence.sentiment.score}")
        #print(f"Sentence sentiment magnitude: {sentence.sentiment.magnitude}")

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    # print(f"Language of the text: {response.language_code}")
    return int(response.document_sentiment.score > 0), response.document_sentiment.score, response.document_sentiment.magnitude

In [114]:
scores = []
magnitudes = []
categories = []
for index, row in imdb_data.iloc[45051:].iterrows():
    category, score, magnitude = sample_analyze_sentiment(row["review"])
    categories.append(category)
    scores.append(score)
    magnitudes.append(magnitude)
    np.save('categories_x.npy', categories)
    np.save('scores_x.npy', scores)
    np.save('magnitudes_x.npy', magnitudes)

In [115]:
categories_1 = np.load("categories_1.npy")
scores_1 = np.load("scores_1.npy")
magnitudes_1 = np.load("magnitudes_1.npy")

categories_2 = np.load("categories_2.npy")
scores_2 = np.load("scores_2.npy")
magnitudes_2 = np.load("magnitudes_2.npy")

categories_3 = np.load("categories_3.npy")
scores_3 = np.load("scores_3.npy")
magnitudes_3 = np.load("magnitudes_3.npy")

In [116]:
categories = np.concatenate((categories_1, categories_2, categories_3), axis=0)
scores = np.concatenate((scores_1, scores_2, scores_3), axis=0)
magnitudes = np.concatenate((magnitudes_1, magnitudes_2, magnitudes_3), axis=0)

In [120]:
len(magnitudes)

50000

In [122]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(imdb_data['sentiment_code'], categories)

In [123]:
accuracy

0.89224